In [57]:
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import QuantumKernel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [58]:
df = pd.read_csv(r"D:\DATA PELATIHAN\Cleaned_data", index_col='ID')

del df['Unnamed: 0']

df

,No_Patient,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
ID,,,,,,,,,,,,,
502,17975,F,50.0,4.700000,46.0,4.900000,4.2,0.9,2.4,1.4,0.500000,24.0,N
735,34221,M,26.0,4.500000,62.0,4.900000,3.7,1.4,1.1,2.1,0.600000,23.0,N
420,47975,F,50.0,4.700000,46.0,4.900000,4.2,0.9,2.4,1.4,0.500000,24.0,N
680,87656,F,50.0,4.700000,46.0,4.900000,4.2,0.9,2.4,1.4,0.500000,24.0,N
504,34223,M,33.0,7.100000,46.0,4.900000,4.9,1.0,0.8,2.0,0.400000,21.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,454316,M,55.0,5.131094,62.0,6.800000,5.3,2.0,1.0,3.5,0.900000,30.1,Y
192,454316,M,55.0,4.800000,88.0,8.284155,5.7,4.0,0.9,3.3,1.800000,30.0,Y
193,454316,M,62.0,6.300000,82.0,6.700000,5.3,2.0,1.0,3.5,1.850595,30.1,Y


In [59]:
# Melakukan standarisasi fitur
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [60]:
# Melakukan encoding label menjadi bilangan bulat
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [61]:
# Membagi dataset menjadi data latihan dan data uji
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

In [63]:
# Menginisialisasi model VQC
# Menginisialisasi objek QuantumKernel
feature_dim = len(train_features[0])
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2)
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)
kernel = QuantumKernel(feature_map, quantum_instance=quantum_instance)

C:\Users\Asani\AppData\Local\Temp\ipykernel_10208\225787259.py:6: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend)


In [64]:
# Menginisialisasi model VQC
vqc = VQC(optimizer=None, feature_map=feature_map, quantum_instance=quantum_instance)

C:\Users\Asani\AppData\Local\Temp\ipykernel_10208\4276740726.py:2: DeprecationWarning: The quantum_instance argument is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the sampler argument.
  vqc = VQC(optimizer=None, feature_map=feature_map, quantum_instance=quantum_instance)


In [65]:
# Melatih model VQC
vqc.fit(train_features, train_labels)

# Memprediksi label dari data uji
predictions = vqc.predict(test_features)

# Melakukan decoding label menjadi nama kelas
predicted_labels = label_encoder.inverse_transform(predictions)

c:\Users\Asani\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Menghitung akurasi prediksi
accuracy = np.mean(predicted_labels == test_labels)
print("Akurasi:", accuracy)